# Simulated Annealing with Custom Weighted Candidate Generator


# This Solution:

- Works best with Sample 0 to 4
- With correct starting text achives 30.x on Sample 5
- @[siavrez](https://www.kaggle.com/siavrez) code that reached 28.5x on Sample 5 can be found here :
  [Solution 2](https://www.kaggle.com/code/siavrez/3rd-place-sa-weighted-candidate-generator-sol2)

# Batch Metric

In [ ]:
"""Evaluation metric for Santa 2024."""

import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ParticipantVisibleError(Exception):
    pass


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
    load_in_8bit: bool = False,
    clear_mem: bool = False,
) -> float:
    """
    Calculates the mean perplexity of submitted text permutations compared to an original text.

    Parameters
    ----------
    solution : DataFrame
        DataFrame containing the original text in a column named 'text'.
        Includes a row ID column specified by `row_id_column_name`.

    submission : DataFrame
        DataFrame containing the permuted text in a column named 'text'.
        Must have the same row IDs as the solution.
        Includes a row ID column specified by `row_id_column_name`.

    row_id_column_name : str
        Name of the column containing row IDs.
        Ensures aligned comparison between solution and submission.

    model_path : str, default='/kaggle/input/gemma-2/transformers/gemma-2-9b/2'
        Path to the serialized LLM.

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    clear_mem : bool, default=False
        Clear GPU memory after scoring by clearing the CUDA cache.
        Useful for testing.

    Returns
    -------
    float
        The mean perplexity score. Lower is better.

    Raises
    ------
    ParticipantVisibleError
        If the submission format is invalid or submitted strings are not valid permutations.

    Examples
    --------
    >>> import pandas as pd
    >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
    >>> solution = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["this is a normal english sentence", "the quick brown fox jumps over the lazy dog"]
    ... })
    >>> submission = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["sentence english normal a is this", "lazy the over jumps fox brown quick the dog"]
    ... })
    >>> score(solution, submission, 'id', model_path=model_path, clear_mem=True) > 0
    True
    """
    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts
    if invalid_mask.any():
        raise ParticipantVisibleError(
            'At least one submitted string is not a valid permutation of the solution string.'
        )

    # Calculate perplexity for the submitted strings
    sub_strings = [
        ' '.join(s.split()) for s in submission['text'].tolist()
    ]  # Split and rejoin to normalize whitespace
    scorer = PerplexityCalculator(
        model_path=model_path,
        load_in_8bit=load_in_8bit,
    )  # Initialize the perplexity calculator with a pre-trained model
    perplexities = scorer.get_perplexity(
        sub_strings
    )  # Calculate perplexity for each submitted string

    if clear_mem:
        # Just move on if it fails. Not essential if we have the score.
        try:
            scorer.clear_gpu_memory()
        except:
            print('GPU memory clearing failed.')

    return float(np.mean(perplexities))


class PerplexityCalculatorBatches:
    """
    Calculates perplexity of text using a pre-trained language model.

    Adapted from https://github.com/asahi417/lmppl/blob/main/lmppl/ppl_recurrent_lm.py

    Parameters
    ----------
    model_path : str
        Path to the pre-trained language model

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    device_map : str, default="auto"
        Device mapping for the model.
    """

    def __init__(
        self,
        model_path: str,
        load_in_8bit: bool = False,
        device_map: str = 'auto',
    ):
        #self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path,padding_side="right")


        # Configure model loading based on quantization setting and device availability
        if load_in_8bit:
            print("Load small)")
            if DEVICE.type != 'cuda':
                raise ValueError('8-bit quantization requires CUDA device')
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,

            )
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
                device_map=device_map,
                #attn_implementation="sdpa",
            )

        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.eval()
    def get_perplexity( self, input_texts: Union[str, List[str]], batch_size: 32) -> Union[float, List[float]]:

        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []

        batches = len(input_texts)//batch_size + (len(input_texts)%batch_size != 0)
        for j in range(batches):

            a = j*batch_size
            b = (j+1)*batch_size
            input_batch = input_texts[a:b]

            with torch.no_grad():

                # Explicitly add sequence boundary tokens to the text
                text_with_special = [f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}" for text in input_batch]

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                    padding=True
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                label = model_inputs['input_ids']
                label[label == self.tokenizer.pad_token_id] = PAD_TOKEN_LABEL_ID

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = label[..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                loss = loss.view(len(logits), -1)
                valid_length = (shift_labels != PAD_TOKEN_LABEL_ID).sum(dim=-1)
                loss = torch.sum(loss, -1) / valid_length

                loss_list += loss.cpu().tolist()

        ppl = [exp(i) for i in loss_list]

        return ppl[0] if single_input else ppl

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        # Delete model and tokenizer if they exist
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'tokenizer'):
            del self.tokenizer

        # Run garbage collection
        gc.collect()
#871 or 178
        # Clear CUDA cache and reset memory stats
        with DEVICE:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()

In [ ]:
scorer = PerplexityCalculatorBatches("/kaggle/input/gemma-2/transformers/gemma-2-9b/2")

# CandidateGenerator

- This class is responsible for generating candidate texts to score.
- We defined different operations (text transformation operations).
- The selection of which operation to use is not purely random, we created a weighting system to give importance to operations based on the score they produce.

In [ ]:
import random
import math
import time
from collections import deque



class CandidateGenerator:
    def __init__(self, scorer, batch_size=32):
        self.global_cache = set()
        self.max_cache_size = 1000000000  # Define max cache size to avoid memory bloat
        self.operations = [ # The selection of operations is based on the sample/ many tires to reach best selections 
            "swap",
            "reverse",
            "removeinsert",
            #"removeinsertn",
            "rotate",
            "shift",
            "swap_adjacent",
            "circular_shift",
            #"reverse_all",
            #"scramble_except_first",
            "block_shift",
            "block_swap",
        ]
        self.operation_weights = [1.0] * len(self.operations)  # Initial equal weights

        # Initialize word importance scores
        self.word_importance = {}
        self.scorer = scorer
        self.batch_size = batch_size

    def compute_word_importance(self, words):
        """
        Compute word importance based on perplexity scores.
        """
        importance_scores = {}
        for word in words:
            # Get perplexity for each word
            perplexity = self.scorer.get_perplexity([word], batch_size=self.batch_size)
            importance_scores[word] = 1 / perplexity[0]  # Higher perplexity -> Lower importance
        self.word_importance = importance_scores

    def targeted_modify_text_batch(self, text, num_modifications):
        words = text.split()
        candidates = []

        # Normalize weights to create a probability distribution
        total_weight = sum(self.operation_weights)
        operation_probs = [w / total_weight for w in self.operation_weights]

        for _ in range(num_modifications):
            operation = random.choices(self.operations, weights=operation_probs, k=1)[0]
            new_words = words[:]

            # Incorporate word importance into operations
            high_importance_words = [
                idx for idx, word in enumerate(new_words) if self.word_importance.get(word, 0) > 0.5
            ]

            if operation == "removeinsert":
                idx1 = random.choice(high_importance_words) if high_importance_words else random.randint(0, len(new_words) - 1)
                word = new_words.pop(idx1)
                idx2 = random.randint(0, len(new_words))
                new_words.insert(idx2, word)
            if operation == "removeinsertn":
                # Determine the number of words to remove and reinsert (N between 1 and 4)
                N = random.randint(1, 4)

                # Ensure N does not exceed the length of the list
                N = min(N, len(new_words))

                for _ in range(N):
                  # Select a word to remove
                  idx1 = random.randint(0, len(new_words) - 1)
                  word = new_words.pop(idx1)

                  # Select a new position to insert
                  idx2 = random.randint(0, len(new_words))
                  new_words.insert(idx2, word)
            elif operation == "swap":
                if high_importance_words:
                    idx1 = random.choice(high_importance_words)
                    idx2 = random.randint(0, len(new_words) - 1)
                else:
                    idx1, idx2 = random.sample(range(len(new_words)), 2)
                new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
            elif operation == "reverse":
                idx1, idx2 = sorted(random.sample(range(len(new_words)), 2))
                new_words[idx1:idx2] = reversed(new_words[idx1:idx2])
            elif operation == "rotate":
                idx1, idx2 = sorted(random.sample(range(len(new_words)), 2))
                if idx2 - idx1 > 1:
                    subarray = new_words[idx1:idx2]
                    rotation = random.randint(1, len(subarray) - 1)
                    new_words[idx1:idx2] = subarray[rotation:] + subarray[:rotation]
            elif operation == "shift":
                idx1, idx2 = sorted(random.sample(range(len(new_words)), 2))
                if idx2 - idx1 > 1:
                    subarray = new_words[idx1:idx2]
                    shift = random.choice([-1, 1])
                    new_words[idx1:idx2] = subarray[shift:] + subarray[:shift]
            elif operation == "swap_adjacent":
                idx = random.randint(0, len(new_words) - 2)
                new_words[idx], new_words[idx + 1] = new_words[idx + 1], new_words[idx]
            elif operation == "circular_shift":
                shift = random.randint(1, len(new_words) - 1)
                new_words = new_words[shift:] + new_words[:shift]
            elif operation == "reverse_all":
                new_words = list(reversed(new_words))
            elif operation == "scramble_except_first":
                first_word = new_words[0]
                rest = new_words[1:]
                random.shuffle(rest)
                new_words = [first_word] + rest
            elif operation == "block_shift":
                idx1, idx2 = sorted(random.sample(range(len(new_words)), 2))
                block = new_words[idx1:idx2]
                del new_words[idx1:idx2]
                insert_idx = random.randint(0, len(new_words))
                new_words[insert_idx:insert_idx] = block
            elif operation == "block_swap":
                idx1, idx2 = sorted(random.sample(range(len(new_words)), 2))
                idx3, idx4 = sorted(random.sample(range(len(new_words)), 2))
                if idx1 < idx2 and idx3 < idx4 and idx1 != idx3:
                    block1 = new_words[idx1:idx2]
                    block2 = new_words[idx3:idx4]
                    new_words[idx1:idx2], new_words[idx3:idx4] = block2, block1

            # Ensure the length of the new text matches the original and is a valid permutation
            if len(new_words) != len(words) or sorted(new_words) != sorted(words):
                continue

            candidate = ' '.join(new_words)

            # Add to global cache if unique
            if candidate not in self.global_cache:
                candidates.append((candidate, operation))  # Track operation for reward-punishment
                self.global_cache.add(candidate)

                # Clear cache if size exceeds limit
                if len(self.global_cache) > self.max_cache_size:
                    self.global_cache.clear()
                    print("Cache Restarted ....")

        #print(f"Current Cache has {len(self.global_cache)}")

        return candidates

    def update_weights(self, operation_scores):
        """
        Updates the weights for operations based on their performance.
        :param operation_scores: A dictionary mapping operations to their scores.
        """
        min_score = min(operation_scores.values())
        max_score = max(operation_scores.values())

        # Reward-punishment mechanism
        for i, operation in enumerate(self.operations):
            score = operation_scores.get(operation, max_score)
            if score == min_score:
                self.operation_weights[i] *= 1.2  # Strong reward for best operation
            else:
                penalty = 0.9 + (0.1 * (score - min_score) / (max_score - min_score + 1e-6))
                self.operation_weights[i] *= penalty

        # Normalize weights to avoid extreme bias
        total_weight = sum(self.operation_weights)
        if total_weight <= 0 or not math.isfinite(total_weight):
            # Reset weights to equal distribution if normalization fails
            self.operation_weights = [1.0] * len(self.operation_weights)
        else:
            self.operation_weights = [w / total_weight for w in self.operation_weights]


# Enhanced Simulated Annealing:

- Tracking operations (text transformation operations score and use that to update the selection of candidates.
- Taking texts with "equal best score" into consideration, this help escape dead-end paths.
  Note: This happens with A100 GPUs at least, where two texts (or more) can have the same score.
- Trying to fix a prefix/postfix part of the text and optimize other parts (didn't really work)
- No improvements: when no improvements happen we reselect a text and start from (basically some shuffling or back to the best text), it's based on experiments.


In [ ]:
class EnhancedSimulatedAnnealing:
    def __init__(self, scorer, initial_text, initial_temperature, cooling_rate, max_iterations, max_time, batch_size=32):
        """
        Initializes the optimizer.
        """
        self.scorer = scorer
        self.initial_words = initial_text.split()
        self.temperature = initial_temperature
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.max_time = max_time
        self.batch_size = batch_size
        self.start_time = None
        self.attractive_results = []

        self.generator = CandidateGenerator(scorer, batch_size)


        # Initialize with the initial solution
        self.current_text = initial_text
        self.current_score = 100000 #self.scorer.get_perplexity(initial_text, 1)
        self.best_text = self.current_text
        self.best_score = self.current_score

        # Maintain a history of scores for adaptive cooling
        self.score_history = deque(maxlen=300)

    def adaptive_cooling(self):
        """
        Adjusts the cooling rate dynamically based on score history.
        """
        if len(self.score_history) < 2:
            return
        volatility = max(self.score_history) - min(self.score_history)
        if volatility < 0.05:  # Low volatility, cool down faster
            self.temperature *= self.cooling_rate * 1.03
        else:  # High volatility, cool down slower
            self.temperature *= self.cooling_rate * 0.97

    def optimize(self):
        """
        Runs the Enhanced Simulated Annealing algorithm with batched neighbor evaluation.
        :return: The optimized text and its perplexity score.
        """
        self.start_time = time.time()
        iteration = 0
        stagnation_limit = 1000
        pre_score = 100000
        results = []

        no_improvement_iterations = 0
        results_file = "ress.csv"


        while iteration < self.max_iterations:
            # Stop if time limit is exceeded
            if time.time() - self.start_time > self.max_time:
                break

            # Generate a batch of candidate solutions
            try:
              candidates_with_operations = self.generator.targeted_modify_text_batch(self.current_text, self.batch_size)
              candidates, candidate_operations = zip(*candidates_with_operations)
            except:
                pass



            # Evaluate perplexity scores for the batch
           # print("Cn size: ", len(candidates))
            #prefix = "from " #the the of and and to in is you that it with as from and have not
            #postfix = " we with not you have bake candy card game chocolate cookie doll eggnog fireplace fruitcake gingerbread holly mistletoe nutcracker ornament ornament peppermint poinsettia puzzle scrooge snowglobe stocking toy unwrap wrapping paper wreath yuletide advent angel beard believe bow carol candle cheer cheer chimney chimney decorations dream drive eat elf family fireplace gifts give greeting grinch holiday hohoho hope jingle jump joy kaggle laugh magi merry milk naughty nice night night peace polar reindeer relax season sing sleigh sleep star visit walk wish wonder workshop workshop"
            #pre = "from the the the of of and and and to is as in that it we with not you have "
            #post = " advent angel beard believe bow candle carol cheer cheer chimney chimney decorations dream drive eat elf family fireplace gifts give greeting grinch holiday hohoho hope jingle jump joy kaggle laugh magi merry milk naughty nice night night peace polar reindeer relax season sing sleigh sleep star visit walk wish wonder workshop workshop"

            #candidates_with_prefix = [pre + candidate+post for candidate in candidates]
            candidates_with_prefix = candidates
            candidate_scores = self.scorer.get_perplexity(candidates_with_prefix, 80) # this number changes based on sample length and GPU
            #print("done")

            # Track operation performance
            operation_scores = {op: float('inf') for op in self.generator.operations}
            for op, score in zip(candidate_operations, candidate_scores):
                operation_scores[op] = min(operation_scores[op], score)

            # Update operation weights
            self.generator.update_weights(operation_scores)

            # Select the best candidate
            best_idx = min(range(len(candidate_scores)), key=lambda i: candidate_scores[i])
            new_text, new_score = candidates[best_idx], candidate_scores[best_idx]


            # Adaptive acceptance
            if new_score <= self.current_score or random.random() < math.exp((self.current_score - new_score) / self.temperature):
                self.current_text = new_text
                self.current_score = new_score

                # Update the best solution if applicable
                if new_score < self.best_score or (new_score == self.best_score and self.best_text != new_text):
                    if (new_score == self.best_score and self.best_text != new_text):
                      print("Please note this is a new text with best_score !")

                    self.best_text = new_text
                    self.best_score = new_score
                    print(f"Iteration {iteration}: New Best Score: {self.best_score}, Text: {self.best_text}")

                    new_result = {
                        "Iteration": iteration,
                        "Current_Score": self.current_score,
                        "Current_Text": self.current_text,
                        }
                    results.append(new_result)
                    pd.DataFrame(results).to_csv(results_file, index=False)
                    no_improvement_iterations = 0
                    # stagnation_limit = stagnation_limit - 100
                    # if stagnation_limit < 100:
                    #   stagnation_limit = 100

            # Update score history and apply adaptive cooling
            self.score_history.append(self.current_score)
            self.adaptive_cooling()

            # Handle stagnation by restarting weights
            if no_improvement_iterations >= stagnation_limit:
                self.generator.operation_weights = [1.0] * len(self.generator.operations)  # Reset weights
                no_improvement_iterations = 0
                words = self.best_text.split()
                # Here we tried diffrent ways to select new seed when no improvments happens
                #mid = len(words)  // random.randint(2, 10)
                #shuffled_part = random.sample(words[mid:], len(words[mid:]))
                #self.current_text = ' '.join(words[:mid] + shuffled_part)
                #shuffled_part =  random.sample(words[:mid], len(words[:mid]))
                #self.current_text = ' '.join(shuffled_part + words[mid:])
                self.current_text = self.best_text
                #self.current_text = reverse_words_order(self.best_text) #' '.join(shuffled_part+words[mid:])
                self.current_score = self.scorer.get_perplexity(self.current_text,1)
                no_improvement_iterations = 0  # Reset stagnation counter
                self.temperature = self.initial_temperature
                #stagnation_limit = stagnation_limit + 100
                pre_score = 100000

            iteration += 1
            new_result = {
                        "Iteration": iteration,
                        "Current_Score": self.current_score,
                        "Current_Text": self.current_text,
                        }
            results.append(new_result)
            pd.DataFrame(results).to_csv(results_file, index=False)


            if iteration % 10 == 0:
                if self.current_score < pre_score:
                  no_improvement_iterations = 0
                  pre_score = self.current_score
                else:
                  no_improvement_iterations += 10
                print(f"Iteration {iteration}: Best Score: {self.best_score}, Current Score and new: {self.current_score} {new_score}, Temperature: {self.temperature} {no_improvement_iterations} {len(candidates)}")
                new_result = {
                    "Iteration": iteration,
                    "Current_Score": self.current_score,
                    "Current_Text": self.current_text,
                  }
                results.append(new_result)
                pd.DataFrame(results).to_csv(results_file, index=False)



        return self.best_text, self.best_score, self.attractive_results



# Initial Text 

What played a good role in achieving better scores was the initial text that we used to start with, after different experiments we found that the following worked best : (Sample IDs from 0 to 5)

- Sorting the text alphabetically : Worked Best with Sample2
- Stopwords sorted then alphabetically sort other words: Gave good starting point for Samples 3 and 4
- Random Starting point (running the code many times in parallel): gave faster convergence and the better result for sample 3
- **Sorted Stopwords, Sorted verbs, Sorted Other words** : Gave best-starting points for Sample 3 and 4 and the most important sample 5 ( even if the starting score is not as low as all the text sorted)

In [ ]:
from nltk.corpus import stopwords

# Load stop words (you might need to download the NLTK stopwords if you haven't already)
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Sort stop words and other words separately
def custom_sort(text):
    words = text.split(" ")
    stop_words_in_text = sorted([word for word in words if word.lower() in stop_words])
    other_words = sorted([word for word in words if word.lower() not in stop_words])
    return " ".join(stop_words_in_text + other_words)


In [ ]:
initial_temperature = 100.0  # Start with a high temperature
cooling_rate = 0.995  # Base cooling rate
max_iterations = 200000  # Allow many iterations
max_time = 480000 # in other words do not stop :-)
batch_size = 5000 # Evaluate up to N candidates in parallel

initial_text = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and"
initial_text = custom_sort(initial_text)
#print(scorer.get_perplexity(initial_text,1))

# optimizer = EnhancedSimulatedAnnealing(
#         scorer,
#         initial_text,
#         initial_temperature,
#         cooling_rate,
#         max_iterations,
#         max_time,
#         batch_size=batch_size,
#     )
# best_text, best_score, best_alternatives = optimizer.optimize()

# #the code really didn't reach here (with high max_time)
# print(f"\nOptimized Text: {best_text}")
# print(f"Best Perplexity Score: {best_score}")
# print(f"Best Alternatives: {best_alternatives}")



In [ ]:
sol = pd.read_csv("/kaggle/input/santa2024-3rd-sol/sol_3rd.csv")
sol.to_csv("submission.csv", index = False)